# With MessageHistory

In the previous content, we introduced how to use RAG to enhance the capabilities of the chat question and answer system.However, if we want to introduce RAG into a conversation with chat memory, so that we can enhance the customization capabilities in a conversation with chat history. In this tutorial, we will be based on the original chat record saved in Sqlite, allowing users to pass in the previous conversation [blog of lilianweng](https://lilianweng.github.io/posts/2023-06-23-agent/) data to expand RAG in chat conversations.About creating a simple conversation chain which uses ConversationEntityMemory backed by a SqliteEntityStore,please see this [doc](https://python.langchain.com/docs/integrations/memory/sqlite).

## Setup

### Dependencies

We’ll use an OpenAI chat model, SQLChatMessageHistory with Sqlite, embeddings and a Chroma vector store in this walkthrough, but everything shown here works with any ChatModel or LLM, Embeddings, and VectorStore or Retriever.

We’ll use the following packages:

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

We need to set environment variable OPENAI_API_KEY, which can be done directly or loaded from a .env file like so:

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# import dotenv
# dotenv.load_dotenv()

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with LangSmith.

Note that LangSmith is not needed, but it is helpful. If you do want to use LangSmith, after you sign up at the link above, make sure to set your environment variables to start logging traces:

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Chat With MessageHisotry Using RAG

### Import Dependencies

In [1]:
import bs4
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from operator import itemgetter

Load the file from [lilianweng's blog](https://lilianweng.github.io/posts/2023-06-23-agent/), do chunk and index

In [ ]:
# Load, chunk and index the contents of the blog.
bs_strainer = bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs_strainer},
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()


def format_docs(docs):
    """
    Format a list of document objects.

    Args:
    - docs (list): A list containing document objects.

    Returns:
    - str: The formatted string where the page_content attribute of each document object is concatenated, separated by two newline characters.
    """
    return "\n\n".join(doc.page_content for doc in docs)

create prompt and add model

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at something. Here is some {context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

create the chain

In [ ]:
context = itemgetter("question") | retriever | format_docs
first_step = RunnablePassthrough.assign(context=context)
chain = first_step | prompt | llm
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///sqlite.db"
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [ ]:
config = {"configurable": {"session_id": "test_session"}}

try the first question

In [ ]:
result = chain_with_history.invoke(
    {"question": "What does the document mainly contain?"}, config
)
print(result)

try the section question

In [ ]:
result = chain_with_history.invoke({"question": "Summarize a title for me"}, config)
print(result)